In [ ]:
import pandas as pd
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', -1)

**Read in file**

In [ ]:
%%time
column_names=['Name', 'ScreenName', 'UserID', 'FollowersCount', 'FriendsCount', 'Location', 'Description', 'CreatedAt', 'StatusID', 'Language', 'Place', 'RetweetCount', 'FavoriteCount', 'Text']
tweets = pd.read_csv('clinton_trump_tweets.txt', sep="\t",  encoding="ISO-8859-1", header=None, names=column_names)
#print initial data
tweets.Location = tweets.Location.astype(str)
tweets.head(20)

**Filter out tweets that start with 'RT'**

In [ ]:
#1.1 filter retweets
tweets = tweets[~tweets.Text.str.startswith("RT")]
tmp = tweets.Text.str.extract(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')

In [ ]:
tmp["Merges"] =  tmp[0]+"://"+tmp[1]+tmp[2]
tmp.Merges = tmp.Merges.astype(str)

In [ ]:
# Not using this since it is very inefficient
# %%time
# def keepHandlesAndMention(text):
#     handlesAndMentions = []
#     words =  text.split()
#     for word in words:
#         if((word.startswith('@') or word.startswith('#')) and len(word) > 1):
#             handlesAndMentions.append(word)
#     return ' '.join(handlesAndMentions)
# tweets['HashMentions'] = tweets.Text.apply(keepHandlesAndMention)

In [ ]:
%%time
def keepHashMentions(text):
    hashMentions = []
    for word in text:
        hashMentions.extend(word)
    return " ".join(hashMentions).strip()
tweets['HashMentions'] = tweets.Text.str.findall('(@\w+)|(#\w+)').apply(keepHashMentions)

**Filter out tweets that have mention/hashtag < 20**

In [ ]:
%%time
# Generate a list of mention/hashes that have a frequency of 20+
top_hash = pd.Series(tweets['HashMentions'].str.cat(sep=' ').split()).value_counts()
top20 = top_hash[top_hash>=20]
top20List = top20.index.tolist()

In [ ]:
%%time
# generate list of mention/hashes that occur 20+ times from our orginal list 
def removeUnder20Mentions(hashMentions):
    mentions =  hashMentions.split()
    mentionsOver20 = []
    for mention in mentions:
        if(mention in top20List):
            mentionsOver20.append(mention)
    return " ".join(mentionsOver20)
tweets['FrequencyOver20'] = tweets.HashMentions.apply(removeUnder20Mentions)


In [ ]:
# remove tweets that don't have any mention/hashes that occure 20+ times
tweets = tweets[tweets.FrequencyOver20 != '']

**Filter out users that have less than 20 tweets**

In [ ]:
#1.2 keep tweets where UID appears 20+ times
tweets = tweets.groupby("UserID").filter(lambda x: len(x) >= 20)

**Plot top 30 locations**

In [ ]:
# Remove locations that are null 
tweets['LocationSplit'] = tweets.Location.apply(lambda x: x.split(',')[0])
# creating new tweets dataframe since we don't want to alter the data for other visualizations
tweetsLocations = tweets[tweets.LocationSplit.notnull()]
tweetsLocations = tweetsLocations[tweetsLocations.LocationSplit != 'nan']
tweetsLocations.LocationSplit = tweetsLocations.LocationSplit.map({'NYC':'New York','New York City':'New York','United States':'USA', "United States of America": 'USA'}).fillna(tweetsLocations.LocationSplit)

In [ ]:
#2.1
tweetsTop30Locations = tweetsLocations[tweetsLocations.LocationSplit.isin(tweetsLocations.LocationSplit.value_counts().nlargest(30).index.tolist())]
tweetsTop30Locations[tweetsTop30Locations.LocationSplit != 'USA'].groupby("LocationSplit").LocationSplit.count().sort_values(ascending=True).plot(kind='barh',figsize=(20,10))

**Make word cloud of the top 3 locations**

In [ ]:
tweetsTop3Locations = tweetsLocations[tweetsLocations.LocationSplit.isin(tweetsLocations.LocationSplit.value_counts().nlargest(3).index.tolist())]
text = " ".join(tweet for tweet in tweetsTop3Locations.Text)

In [ ]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["way","back","null","en","say","new","go","ye","thing","well","big","us","great","https", "still", "need", "co", "one","will","Thank","know","going","lol","good", "take","even","really","now"])
stopwords.update(tmp.Merges.unique().flatten())

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords).generate(text)

# Display the generated image:
plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

**Plot number of tweets of top 50 users**

In [ ]:
tweetsTop50Users = tweets[tweets.UserID.isin(tweets.UserID.value_counts().nlargest(50).index.tolist())]

In [ ]:
tweetsTop50Users.groupby("ScreenName").ScreenName.count().sort_values().plot(kind='barh',figsize=(20,10))

**Word cloud of top 3 users**

In [ ]:
tweetsTop3Users = tweets[tweets.UserID.isin(tweets.UserID.value_counts().nlargest(3).index.tolist())]
top3UsersText = " ".join(" ".join(tweet.split()) for tweet in tweetsTop3Users.Text)

In [ ]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["YOUNGMA", "https", "co", "DOWNLOAD", "CJRr8Y9xHM"])
stopwords.update(tmp.Merges.unique().flatten())

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords).generate(top3UsersText)

# Display the generated image:
plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

**Top 100 mentions/hashtags**

In [ ]:
top_hash.nlargest(100).sort_values(ascending=True).plot(kind='barh',figsize=(50,20))
text = " ".join(tweet for tweet in tweets.HashMentions)

In [ ]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["YouTube","Now","DOWNLOAD","c0nvey","ALBUM","YOUNGMA","Cub","DOWNLOAD", "SPOTIFY","quote"])

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords).generate(text)

# Display the generated image:
plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()